#Question number 1

In [1]:
from keras import models
from keras import layers
from keras.layers import Input, Dense, Lambda, MaxPooling1D, Conv1D, Flatten, LSTM
import keras.backend as K
from keras.models import Model,Sequential
from keras import Input

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type

In [2]:

import pickle


with open('hw4_trs.pkl', 'rb') as f:
    data_train = pickle.load(f)
    
    
import numpy as np
import librosa


train_data=[]

for i in range(500):
    S=librosa.stft(data_train[i], n_fft=1024, hop_length=512)
    train_data.append(S)
print(S.shape)
train_data=np.stack(train_data)
print(train_data.shape)

test_data=[]
with open('hw4_tes.pkl', 'rb') as f1:
    data_test = pickle.load(f1)

    
for i in range(200):
    S=librosa.stft(data_test[i], n_fft=1024, hop_length=512)
    test_data.append(S)
print(len(test_data))

test_data=np.stack(test_data)
print(test_data.shape)

(513, 32)
(500, 513, 32)
200
(200, 513, 45)


In [3]:
train_data=np.abs(train_data).transpose((0,2,1))
print(train_data.shape)

(500, 32, 513)


In [4]:
test_data=np.abs(test_data).transpose((0,2,1))
print(test_data.shape)

(200, 45, 513)


In [5]:
#Siamese Network
## Reference was taken from keras Documentation and stack overflow
# Define the tensors for the two input speech signals
first_In = Input(shape=(None,513))
second_In = Input(shape=(None,513))
    
# Convolutional Neural Network
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2,activation='relu',strides=1, padding='same',input_shape=(None,513),kernel_initializer='he_normal'))
model.add(MaxPooling1D(pool_size=2,strides=1, padding='valid'))
model.add(Conv1D(filters=32, kernel_size=2,activation='relu',strides=1, padding='same',kernel_initializer='he_normal'))
model.add(MaxPooling1D(pool_size=2,strides=1, padding='valid'))
model.add(Conv1D(filters=32, kernel_size=3,activation='relu',strides=1, padding='same',kernel_initializer='he_normal'))
model.add(MaxPooling1D(pool_size=2,strides=1, padding='valid'))
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', kernel_initializer='he_normal' ))
model.add(LSTM(412))
    
# Generate the encodings (feature vectors) for the two speech signals
encoded_1 = model(first_In)
encoded_2 = model(second_In)
    
 # Add a customized layer to compute the absolute difference between the encodings
L1_layer = Lambda(lambda tensors:K.sum(tensors[0]*tensors[1],axis=-1,keepdims=True)) 
L1_dot = L1_layer([encoded_1, encoded_2])
    
# Add a dense layer with a sigmoid unit to generate the similarity score
prediction = Dense(1,activation='sigmoid')(L1_dot)
    
# Connect the inputs with the outputs
siamese_net = Model(inputs=[first_In,second_In],outputs=prediction)
    
# compile the model
siamese_net.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

   

In [6]:
#Generating batches of samples of train data
from itertools import combinations
import random
import itertools
def generator(train_data):
    train_data=np.abs(train_data).transpose((0,2,1))
    while True:
        for i in range(0,510,10):
            if i>=len(train_data)-10:
                i=0
            p=np.array(list(combinations(train_data[i:i+10],2)))
            negative_1=list(itertools.product(train_data[i:i+10],train_data[i+10:500]))
            negative_1=random.sample(negative_1,45)
            negative_1=np.array(negative_1)
            data_List=np.concatenate([p, negative_1])
            label=[]
            for i in range(45): 
                
                label.append(1)
            for i in range(45):
                label.append(0)
        
            c = list(zip(data_List, label))
            random.shuffle(c)
            data_List, label = zip(*c)
            data_List=np.array(data_List)
            label=np.array(label)
            data_List=data_List.transpose((1,0,2,3))

            yield [data_List[0],data_List[1]], label[:90]

In [7]:
#Generating batches of samples of test data
from itertools import combinations
import random
import itertools
def generator1(test_data):
    test_data=np.abs(test_data).transpose((0,2,1))
    while True:
        for i in range(0,210,10):
            if i>=len(test_data)-10:
                i=0
            p1=np.array(list(combinations(test_data[i:i+10],2)))
            negative_2=list(itertools.product(test_data[i:i+10],test_data[i+10:200]))
            negative_2=random.sample(negative_2,45)
            negative_2=np.array(negative_2)
            data_List1=np.concatenate([p1, negative_2])
            label1=[]
            for i in range(45):
                label1.append(1)
            for i in range(45):
                 label1.append(0)
            c1 = list(zip(data_List1, label1))
            random.shuffle(c1)
            data_List1, label1 = zip(*c1)
            data_List1=np.array(data_List1)
            label1=np.array(label1)
            data_List1=data_List1.transpose((1,0,2,3))

            yield [data_List1[0],data_List1[1]], label1[:90]


In [8]:
# Training model
siamese_net.fit_generator(generator(train_data.transpose((0,2,1))),steps_per_epoch = len(train_data)/10, epochs = 46)

Epoch 1/46
50/50 [==============================] - 17s 343ms/step - loss: 0.8624 - acc: 0.5284
Epoch 2/46
50/50 [==============================] - 18s 354ms/step - loss: 0.6896 - acc: 0.5604
Epoch 3/46


KeyboardInterrupt: 

In [ ]:
print("Loss and Accuracy")
print(siamese_net.evaluate_generator(generator1(test_data.transpose((0,2,1))),steps=20))

In [ ]:
for i in range(100):
    print(siamese_net.evaluate_generator(generator1(test_data.transpose((0,2,1))),steps=20))
    

[0.5487573377788066, 0.7566666692495346]
[0.5264871641993523, 0.7633333295583725]
[0.5284012071788311, 0.7649999976158142]
[0.5365496590733528, 0.7666666626930236]
[0.5503831684589386, 0.7572222203016281]
[0.5395398572087288, 0.7616666615009308]
[0.5279035538434982, 0.7616666674613952]
[0.5462457910180092, 0.7594444364309311]
[0.5402844786643982, 0.7572222232818604]
[0.5592226281762123, 0.7588888943195343]
[0.5583599336445332, 0.7533333331346512]
[0.5379051901400089, 0.7594444453716278]
[0.5240912184119224, 0.7677777737379075]
[0.49870396628975866, 0.774444442987442]
[0.5237208358943463, 0.7688888847827912]
[0.5572159722447395, 0.7538888841867447]
[0.5554444551467895, 0.7544444501399994]
[0.510335161536932, 0.778333330154419]
[0.548049197345972, 0.7516666650772095]
[0.5270741648972035, 0.7644444465637207]
[0.5424028195440769, 0.7588888883590699]
[0.5603345267474651, 0.7527777791023255]
[0.5477474495768547, 0.7577777743339539]
[0.5437770634889603, 0.7655555516481399]
[0.5425609283149242